In [1]:
import torch
import torch.nn as nn
import numpy as np
from scipy.stats import multivariate_normal
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


In [2]:
import wandb

project_name = "Time-GAN"

wandb.init(project=project_name)
wandb.run.name = "TEST"
wandb.run.save()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: se99an. Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [3]:
class Embedder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(Embedder, self).__init__()
        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, x):
        r_out, _ = self.rnn(x)
        output = torch.sigmoid(self.fc(r_out))
        return output


class Recovery(nn.Module):
    def __init__(self, hidden_dim, output_dim, num_layers):
        super(Recovery, self).__init__()
        self.rnn = nn.GRU(hidden_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, h):
        r_out, _ = self.rnn(h)
        output = torch.sigmoid(self.fc(r_out))
        return output

class Generator(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(Generator, self).__init__()
        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, z):
        r_out, _ = self.rnn(z)
        output = torch.sigmoid(self.fc(r_out))
        return output


class Supervisor(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(Supervisor, self).__init__()
        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers-1, batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim)

    def forward(self, h):
        r_out, _ = self.rnn(h)
        output = torch.sigmoid(self.fc(r_out))
        return output
    
class Discriminator(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(Discriminator, self).__init__()
        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, 1)

    def forward(self, h):
        r_out, _ = self.rnn(h)
        output = torch.sigmoid(self.fc(r_out))
        return output



In [4]:
import wandb  # Ensure you've initialized wandb and logged in appropriately.

def train_GAN(num_epochs_embedding, num_epochs_supervised, num_epochs_adversarial, data_loader, batch_size, seq_len, hidden_dim, embedder, recovery, generator, supervisor, discriminator, E_optimizer, G_optimizer, D_optimizer):
    # Embedding Phase: Training Embedder and Recovery
    for epoch in range(num_epochs_embedding):
        for X_mb, T_mb in data_loader:  # Iterate through mini-batches from the data loader.
            H_mb = embedder(X_mb)
            X_tilde = recovery(H_mb)
            E_loss = torch.mean((X_mb - X_tilde)**2)  # MSE loss

            E_optimizer.zero_grad()
            E_loss.backward()
            E_optimizer.step()
            wandb.log({"Embedding Loss": E_loss})
            
    # Supervised Training Phase
    # Here, the intention was to train the generator with supervised loss,
    # but it mistakenly updates using the embedder optimizer.
    # Adjusting to reflect the correct training intention.
    for epoch in range(num_epochs_supervised):
        for X_mb, T_mb in data_loader:
            Z_mb = torch.randn([batch_size, seq_len, hidden_dim])
            H_mb = embedder(X_mb)
            E_hat = generator(Z_mb)
            H_hat_supervised = supervisor(E_hat)  # Generate supervised embeddings
            S_loss = torch.mean((H_mb - H_hat_supervised)**2)  # Supervised loss for generator

            G_optimizer.zero_grad()
            S_loss.backward()
            G_optimizer.step()
            wandb.log({"Supervised Loss": S_loss})
            
    # Adversarial Training Phase
    for epoch in range(num_epochs_adversarial):
        for X_mb, T_mb in data_loader:
            # Discriminator Training
            for _ in range(2):  # Typically, discriminator is trained more frequently.
                Z_mb = torch.randn([batch_size, seq_len, hidden_dim])
                H_mb = embedder(X_mb)
                H_fake = generator(Z_mb).detach()  # Detach to avoid training generator here.

                D_real = discriminator(H_mb)
                D_fake = discriminator(H_fake)

                D_loss = -torch.mean(torch.log(D_real + 1e-8) + torch.log(1 - D_fake + 1e-8))

                D_optimizer.zero_grad()
                D_loss.backward()
                D_optimizer.step()
                wandb.log({"Discriminator Loss": D_loss})
                
            # Generator Training
            Z_mb = torch.randn([batch_size, seq_len, hidden_dim])
            H_fake = generator(Z_mb)

            D_fake = discriminator(H_fake)
            G_loss = -torch.mean(torch.log(D_fake + 1e-8))

            G_optimizer.zero_grad()
            G_loss.backward()
            G_optimizer.step()
            wandb.log({"Generator Loss": G_loss})
            
    return embedder, recovery, generator, supervisor, discriminator


In [5]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
def generate_synthetic_time_series(num_samples, seq_length, freq=1, noise_level=0.1, trend=False):
    """
    Generates synthetic time series data.
    
    Parameters:
    - num_samples: Number of time series samples to generate.
    - seq_length: Length of each time series.
    - freq: Frequency of the sine wave.
    - noise_level: Standard deviation of Gaussian noise added to the data.
    - trend: If True, adds a linear trend to the data.
    
    Returns:
    - data: Generated synthetic time series data of shape (num_samples, seq_length).
    """
    time = np.linspace(0, 2 * np.pi, seq_length)
    sine_wave = np.sin(freq * time)
    
    if trend:
        trend = np.linspace(0, 1, seq_length)
        sine_wave += trend
    
    data = np.zeros((num_samples, seq_length))
    for i in range(num_samples):
        noise = np.random.normal(0, noise_level, seq_length)
        data[i, :] = sine_wave + noise
    
    return data

def create_dataloader(data, batch_size=64):
    """
    Creates a DataLoader from the given time series data.
    
    Parameters:
    - data: Time series data of shape (num_samples, seq_length).
    - batch_size: Size of each batch.
    
    Returns:
    - DataLoader object.
    """
    # Convert numpy array to PyTorch tensor
    data_tensor = torch.tensor(data, dtype=torch.float32)
    
    # Create TensorDataset
    dataset = TensorDataset(data_tensor, data_tensor)  # Targets are same as inputs for generative models
    
    # Create DataLoader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    return dataloader

# Generate synthetic time series data
num_samples = 1000  # Total number of time series samples
seq_length = 100  # Length of each time series
batch_size = 64  # Batch size for training
data = generate_synthetic_time_series(num_samples, seq_length, freq=2, noise_level=0.2, trend=True)

In [6]:
# 데이터를 PyTorch 텐서로 변환하고, 마지막 차원을 추가합니다.
data_tensor = torch.tensor(data, dtype=torch.float32).unsqueeze(-1)

# TensorDataset과 DataLoader 생성
dataset = TensorDataset(data_tensor, data_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)


In [7]:
# 모델 파라미터 설정
input_dim = 1
hidden_dim = 24
output_dim = 1
num_layers = 3


# 모델 인스턴스화
embedder = Embedder(input_dim, hidden_dim, num_layers)
recovery = Recovery(hidden_dim, output_dim, num_layers)
generator = Generator(hidden_dim, hidden_dim, num_layers)
supervisor = Supervisor(hidden_dim, hidden_dim, num_layers-1)
discriminator = Discriminator(hidden_dim, hidden_dim, num_layers)

In [8]:
# 옵티마이저
E_optimizer = optim.Adam(list(embedder.parameters()) + list(recovery.parameters()), lr=0.001)
G_optimizer = optim.Adam(list(generator.parameters()) + list(supervisor.parameters()), lr=0.001)
D_optimizer = optim.Adam(discriminator.parameters(), lr=0.001)

# 훈련
num_epochs_embedding = 10
num_epochs_adversarial = 10
num_epochs_supervised = 10
train_GAN(num_epochs_embedding, num_epochs_adversarial, num_epochs_supervised, dataloader, batch_size, seq_length, hidden_dim, embedder, recovery, generator, supervisor, discriminator, E_optimizer, G_optimizer, D_optimizer)

(Embedder(
   (rnn): GRU(1, 24, num_layers=3, batch_first=True)
   (fc): Linear(in_features=24, out_features=24, bias=True)
 ),
 Recovery(
   (rnn): GRU(24, 24, num_layers=3, batch_first=True)
   (fc): Linear(in_features=24, out_features=1, bias=True)
 ),
 Generator(
   (rnn): GRU(24, 24, num_layers=3, batch_first=True)
   (fc): Linear(in_features=24, out_features=24, bias=True)
 ),
 Supervisor(
   (rnn): GRU(24, 24, batch_first=True)
   (fc): Linear(in_features=24, out_features=24, bias=True)
 ),
 Discriminator(
   (rnn): GRU(24, 24, num_layers=3, batch_first=True, bidirectional=True)
   (fc): Linear(in_features=48, out_features=1, bias=True)
 ))

In [9]:
generated_samples = []
for _ in range(10):
    Z_mb = torch.randn([batch_size, seq_length, hidden_dim])
    H_fake = generator(Z_mb)
    generated_samples.append(H_fake.detach().numpy())
    
generated_samples = np.array(generated_samples)

In [10]:
# model params 저장
wandb.watch(embedder)
wandb.watch(recovery)
wandb.watch(generator)
wandb.watch(supervisor)
wandb.watch(discriminator)

wandb.save("model.h5")
wandb.save("model.pt")
wandb.finish()

Discriminator Loss,▃▄▃▂▁▁█▇▄▃▃▂▂▁▁▁▁▂▆▆▅▄▃▃▃▃▃▃▃▃▂▂▁▁▁▁▆▂▂▇
Embedding Loss,████████▇▇▇▆▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Generator Loss,▂▂▂▄▇█▆▅▃▂▂▃▄▅▆▅▆▅▁▁▁▂▂▂▂▂▂▂▂▂▃▄▅▅▆▆▇▆▇▇
Supervised Loss,█▇▆▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▄▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁
Discriminator Loss,2.67575
Embedding Loss,0.12419
Generator Loss,2.7333
Supervised Loss,0.00139


In [11]:
# generated_samples 저장
import os
os.makedirs("generated_samples", exist_ok=True)
np.save("generated_samples/generated_samples.npy", generated_samples)



# 모델 저장
os.makedirs("models", exist_ok=True)
torch.save(embedder.state_dict(), "models/embedder.pt")
torch.save(recovery.state_dict(), "models/recovery.pt")
torch.save(generator.state_dict(), "models/generator.pt")
torch.save(supervisor.state_dict(), "models/supervisor.pt")
torch.save(discriminator.state_dict(), "models/discriminator.pt")



In [12]:
# 모델 로드
embedder = Embedder(input_dim, hidden_dim, num_layers)
recovery = Recovery(hidden_dim, output_dim, num_layers)
generator = Generator(hidden_dim, hidden_dim, num_layers)
supervisor = Supervisor(hidden_dim, hidden_dim, num_layers-1)
discriminator = Discriminator(hidden_dim, hidden_dim, num_layers)